In [1]:
import pdfplumber

In [27]:
pdf_path = './data/train_source/「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》.pdf'

def is_within_table(word_bbox, table_areas):
    for table_bbox in table_areas:
        if (word_bbox[0] >= table_bbox[0] and word_bbox[2] <= table_bbox[2] and
            word_bbox[1] >= table_bbox[1] and word_bbox[3] <= table_bbox[3]):
            return True
    return False

def extract_word_and_tables(page):

    tables = page.find_tables()
    table_areas = [table.bbox for table in tables]

    items = []

    words = page.extract_words()

    cnt = 0

    for word_info in words:
        word_bbox = (word_info['x0'], word_info['top'], word_info['x1'], word_info['bottom'])

        if not is_within_table(word_bbox, table_areas):
            # print(word_info['text'], cnt)
            items.append(('text', word_bbox, word_info['text'], cnt))
        else:
            pass
            # items.append(('table_text', word_bbox, word_info['text']))
        cnt += 1

    # 테이블 추출 및 위치 기록
    for table in tables:
        items.append(('table', table.bbox, table.extract(), None))

    # y좌표, cnt 기준 정렬
    items.sort(key=lambda x: (x[1][1], x[3]))

    # None나오기 전까지 cnt로 다시 정렬 -> y좌표가 애매할 때가 있어서 순서 섞임
    sorted_items = []
    start_idx = 0

    for idx, item in enumerate(items):
        if item[3] is None:
            # None이 나오기 전까지의 부분 items를 x[3](cnt)에 대해 정렬
            sorted_items.extend(sorted(items[start_idx:idx], key=lambda x: x[3]))
            # None인 원소는 그대로 추가
            sorted_items.append(item)
            start_idx = idx + 1

    # 마지막 구간 정렬
    if start_idx < len(items):
        sorted_items.extend(sorted(items[start_idx:], key=lambda x: x[3]))

    for item in sorted_items:
        if item[0] == 'text':
            print(item[2], end=' ')
        elif item[0] == 'table':
            print("\n[Table]")
            for row in item[2]:
                print(row)
            print("\n")
        elif item[0] == 'table_text':
            pass
    print("\n")

# PDF 파일 열기
with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages):

        page_width = page.width

        if page_width > 600:
            mid_x = page_width / 2
            
            tables = page.find_tables()
            table_areas = [table.bbox for table in tables]

            words_and_tables = []

            left_clip = (0, 0, mid_x, page.height)
            right_clip = (mid_x, 0, page_width, page.height)

            left_page = page.within_bbox(left_clip)
            right_page = page.within_bbox(right_clip)
            print(f'{page_num} left page: ')
            extract_word_and_tables(left_page)
            print(f'{page_num} right page: ')
            extract_word_and_tables(right_page)
        else:
            print(f'{page_num} less than 600pt: ')
            extract_word_and_tables(page)

0 less than 600pt: 

[Table]
['', '']
[None, '']



[Table]
['']
['23-1\n2023.']




1 left page: 
FIS ISSUE & FOCUS 조세지출 연계관리 조세지출 관리는 매번 정부가 바뀔 때마다 정부 의제로 부각된다. 그런데 그 의제는 이내 흐지부지되고 만다. (Tax Expenditures Management) 조세지출 관리는 재정의 한 축으로 지속가능한 재정운영 차원에서 중요한 이슈인데도 재정지출에 비해 상대적 으로 관리가 느슨하다. 즉, 재정지출은 비교적 체계적으로 관리가 되고 있지만 조세지출은 상대적으로 덜 체계적으로 관리되고 있다. 또, 재정지출에 비해 정치적 영향으로부터 자유롭지도 못하다. 이 글을 통해 조세지출을 재정지출과 동일하게 인식해 관리 되어야 함은 물론 조세지출과 재정지출을 체계적으로 연계해 관리할 필요성이 있음을 알리고자 한다. 또한 독자가 조세지출 관리의 중요성에 대해 새롭게 인식할 수 있는 계기가 되길 기대한다. 01 들어가며 ISSUE 왜 조세지출 관리에 주목해야 하는가? 재정지출과조세지출의연계관리는중요한의제,재정지출규모가증가추세에있고,조세지출규모마저 큰폭으로증가추세에있기때문 조세지출관리상제약,두재정지출모두정부의재정부담으로작용하지만,재정·조세지출을포괄한총량 관리체계가부재,통합적검토미흡 예산안부속서인조세지출예산서에분야별조세지출총량을제시하나재정지출과연계한총량관리는 미흡 FOCUS 조세지출 관련 주요 쟁점은? 조세지출(조세특례성과평가)과재정지출이별개의제도로관리되고있어통합재정관리측면에서한계가 있으며통합재정지출관점에서재정지출과조세지출을연계해관리할필요 조세지출DB관리,재정지출과유사한조세지출의통합심층평가,재정총량관리관점에서한도제및 PAYGO(PayAsYouGO)적용을고려 예산안부속서류인‘조세지출예산서’와조세지출평가(성과관리)